In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.svm import OneClassSVM

In [2]:
# load data
data_raw = pd.read_csv('C:/Data/DataCoSupplyChainDataset.csv', encoding = "ISO-8859-1")

# load data description
data_description = pd.read_csv('C:/Data/DescriptionDataCoSupplyChain.csv', encoding = "ISO-8859-1")

In [3]:
data_raw.select_dtypes(include = ["object"]).columns

Index(['Type', 'Delivery Status', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Department Name', 'Market',
       'Order City', 'Order Country', 'order date (DateOrders)',
       'Order Region', 'Order State', 'Order Status', 'Product Image',
       'Product Name', 'shipping date (DateOrders)', 'Shipping Mode'],
      dtype='object')

In [4]:
data_raw.select_dtypes(include = ["object"])

,Type,Delivery Status,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Lname,Customer Password,Customer Segment,...,Order City,Order Country,order date (DateOrders),Order Region,Order State,Order Status,Product Image,Product Name,shipping date (DateOrders),Shipping Mode
0,DEBIT,Advance shipping,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,Holloway,XXXXXXXXX,Consumer,...,Bekasi,Indonesia,1/31/2018 22:56,Southeast Asia,Java Occidental,COMPLETE,http://images.acmesports.sports/Smart+watch,Smart watch,2/3/2018 22:56,Standard Class
1,TRANSFER,Late delivery,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,Luna,XXXXXXXXX,Consumer,...,Bikaner,India,1/13/2018 12:27,South Asia,Rajastán,PENDING,http://images.acmesports.sports/Smart+watch,Smart watch,1/18/2018 12:27,Standard Class
2,CASH,Shipping on time,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,Maldonado,XXXXXXXXX,Consumer,...,Bikaner,India,1/13/2018 12:06,South Asia,Rajastán,CLOSED,http://images.acmesports.sports/Smart+watch,Smart watch,1/17/2018 12:06,Standard Class
3,DEBIT,Advance shipping,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,Tate,XXXXXXXXX,Home Office,...,Townsville,Australia,1/13/2018 11:45,Oceania,Queensland,COMPLETE,http://images.acmesports.sports/Smart+watch,Smart watch,1/16/2018 11:45,Standard Class
4,PAYMENT,Advance shipping,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,Hendricks,XXXXXXXXX,Corporate,...,Townsville,Australia,1/13/2018 11:24,Oceania,Queensland,PENDING_PAYMENT,http://images.acmesports.sports/Smart+watch,Smart watch,1/15/2018 11:24,Standard Class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,Shipping on time,Fishing,Brooklyn,EE. UU.,XXXXXXXXX,Maria,Peterson,XXXXXXXXX,Home Office,...,Shanghái,China,1/16/2016 3:40,Eastern Asia,Shanghái,CLOSED,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,1/20/2016 3:40,Standard Class
180515,DEBIT,Late delivery,Fishing,Bakersfield,EE. UU.,XXXXXXXXX,Ronald,Clark,XXXXXXXXX,Corporate,...,Hirakata,Japón,1/16/2016 1:34,Eastern Asia,Osaka,COMPLETE,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,1/19/2016 1:34,Second Class
180516,TRANSFER,Late delivery,Fishing,Bristol,EE. UU.,XXXXXXXXX,John,Smith,XXXXXXXXX,Corporate,...,Adelaide,Australia,1/15/2016 21:00,Oceania,Australia del Sur,PENDING,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,1/20/2016 21:00,Standard Class
180517,PAYMENT,Advance shipping,Fishing,Caguas,Puerto Rico,XXXXXXXXX,Mary,Smith,XXXXXXXXX,Consumer,...,Adelaide,Australia,1/15/2016 20:18,Oceania,Australia del Sur,PENDING_PAYMENT,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,1/18/2016 20:18,Standard Class


In [5]:
# removing redundant categorical features for modeling
remove_cat = ['Category Name', 'Customer City', 
               'Customer Country', 'Customer Email', 
               'Customer Fname', 'Customer Lname', 
               'Customer Password', 'Customer State',
               'Customer Street', 'Department Name',
               'Market', 'Order City', 'Order Country',
               'Order Region', 'Order State', 'Product Image',
               'Product Name']

In [6]:
data_raw.select_dtypes(exclude = ["object"]).columns

Index(['Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Late_delivery_risk',
       'Category Id', 'Customer Id', 'Customer Zipcode', 'Department Id',
       'Latitude', 'Longitude', 'Order Customer Id', 'Order Id',
       'Order Item Cardprod Id', 'Order Item Discount',
       'Order Item Discount Rate', 'Order Item Id', 'Order Item Product Price',
       'Order Item Profit Ratio', 'Order Item Quantity', 'Sales',
       'Order Item Total', 'Order Profit Per Order', 'Order Zipcode',
       'Product Card Id', 'Product Category Id', 'Product Description',
       'Product Price', 'Product Status'],
      dtype='object')

In [7]:
data_raw.select_dtypes(exclude = ["object"])

,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Late_delivery_risk,Category Id,Customer Id,Customer Zipcode,Department Id,Latitude,...,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Price,Product Status
0,3,4,91.250000,314.640015,0,73,20755,725.0,2,18.251453,...,1,327.750000,314.640015,91.250000,NaN,1360,73,NaN,327.750000,0
1,5,4,-249.089996,311.359985,1,73,19492,725.0,2,18.279451,...,1,327.750000,311.359985,-249.089996,NaN,1360,73,NaN,327.750000,0
2,4,4,-247.779999,309.720001,0,73,19491,95125.0,2,37.292233,...,1,327.750000,309.720001,-247.779999,NaN,1360,73,NaN,327.750000,0
3,3,4,22.860001,304.809998,0,73,19490,90027.0,2,34.125946,...,1,327.750000,304.809998,22.860001,NaN,1360,73,NaN,327.750000,0
4,2,4,134.210007,298.250000,0,73,19489,725.0,2,18.253769,...,1,327.750000,298.250000,134.210007,NaN,1360,73,NaN,327.750000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,4,4,40.000000,399.980011,0,45,1005,11207.0,7,40.640930,...,1,399.980011,399.980011,40.000000,NaN,1004,45,NaN,399.980011,0
180515,3,2,-613.770019,395.980011,1,45,9141,93304.0,7,35.362545,...,1,399.980011,395.980011,-613.770019,NaN,1004,45,NaN,399.980011,0
180516,5,4,141.110001,391.980011,1,45,291,6010.0,7,41.629959,...,1,399.980011,391.980011,141.110001,NaN,1004,45,NaN,399.980011,0
180517,3,4,186.229996,387.980011,0,45,2813,725.0,7,18.213350,...,1,399.980011,387.980011,186.229996,NaN,1004,45,NaN,399.980011,0


In [8]:
# removing redundant continuous features for modeling
remove_cts = ['Category Id', 'Customer Id', 'Customer Zipcode',
               'Department Id', 'Customer Id', 'Order Id', 
               'Order Item Cardprod Id', 'Order Item Id',
               'Order Zipcode', 'Product Card Id', 'Product Category Id',
               'Product Description', 'Product Status', 'Latitude', 'Longitude', 
               'Order Customer Id']

In [ ]:
remove_list = remove_cat + remove_cts
data_raw.drop(remove_list, axis=1, inplace=True)

In [12]:
# check the dimension of the data
data_raw.shape
data_select = data_raw.select_dtypes(include='float')

In [13]:
# Function to calculate Z-scores and identify outliers for each column
def detect_outliers_zscore(df, threshold=3):
    outlier_summary = pd.DataFrame(index=df.index)
    outlier_summary['Outlier_Count'] = 0
    
    for column in df.columns:
        df[f'{column}_Z-Score'] = (df[column] - df[column].mean()) / df[column].std()
        df[f'{column}_Outlier'] = df[f'{column}_Z-Score'].apply(lambda x: 'Yes' if abs(x) > threshold else 'No')
        
        # Add to the count of outliers per row
        outlier_summary['Outlier_Count'] += (df[f'{column}_Outlier'] == 'Yes').astype(int)
    
    # Calculate the overall percentage of outliers
    total_outliers = (outlier_summary['Outlier_Count'] > 0).sum()
    outlier_percentage = (total_outliers / len(df)) * 100
    
    print(f"\nPercentage of rows with at least one outlier identified: {outlier_percentage:.2f}%")
    
    return df, outlier_summary

# Run outlier detection
df_with_outliers, outlier_summary = detect_outliers_zscore(data_select)

# Print summary of the outliers
print(outlier_summary)


Percentage of rows with at least one outlier identified: 4.77%
        Outlier_Count
0                   0
1                   0
2                   0
3                   0
4                   0
...               ...
180514              0
180515              3
180516              0
180517              0
180518              0

[180519 rows x 1 columns]


In [16]:
data_select = data_raw.select_dtypes(include='float')
def detect_outliers_iqr(df):
    outlier_summary = pd.DataFrame(index=df.index)
    outlier_summary['Outlier_Count'] = 0
    
    for column in df.columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identify outliers
        df[f'{column}_Outlier'] = df[column].apply(lambda x: 'Yes' if x < lower_bound or x > upper_bound else 'No')
        
        # Add to the count of outliers per row
        outlier_summary['Outlier_Count'] += (df[f'{column}_Outlier'] == 'Yes').astype(int)
    
    # Calculate the overall percentage of rows with at least one outlier
    total_outliers = (outlier_summary['Outlier_Count'] > 0).sum()
    outlier_percentage = (total_outliers / len(df)) * 100
    
    print(f"\nPercentage of rows with at least one outlier identified: {outlier_percentage:.2f}%")
    
    return df, outlier_summary

# Run IQR-based outlier detection
float_df_with_outliers, outlier_summary = detect_outliers_iqr(data_select)

# Print the DataFrame with outlier flags
print(outlier_summary)


Percentage of rows with at least one outlier identified: 16.64%
        Outlier_Count
0                   0
1                   3
2                   3
3                   0
4                   0
...               ...
180514              0
180515              3
180516              0
180517              2
180518              2

[180519 rows x 1 columns]


In [20]:
data_select = data_raw.select_dtypes(include='float')

# Fit the Isolation Forest model
iso_forest = IsolationForest(contamination=0.1, random_state=42)
data_select['Outlier_Prediction'] = iso_forest.fit_predict(data_select)

# Mark outliers
data_select['Outlier'] = data_select['Outlier_Prediction'].apply(lambda x: 'Yes' if x == -1 else 'No')

# Calculate the overall percentage of outliers
outlier_count = (data_select['Outlier'] == 'Yes').sum()
outlier_percentage = (outlier_count / len(data_select)) * 100

print(f"\nPercentage of outliers identified: {outlier_percentage:.2f}%")


Percentage of outliers identified: 9.98%


In [23]:
data_select = data_raw.select_dtypes(include='float')

# Initialize the DBSCAN model
dbscan = DBSCAN(eps=3, min_samples=5)  # You may need to tune `eps` and `min_samples` for your data

# Fit the model and predict clusters
labels = dbscan.fit_predict(data_select)

# Add the outlier information to the DataFrame
# In DBSCAN, label `-1` indicates noise (outliers)
data_select['Outlier'] = np.where(labels == -1, 'Yes', 'No')

# Calculate the overall percentage of outliers
outlier_percentage = (data_select['Outlier'] == 'Yes').mean() * 100
print(f"\nPercentage of outliers identified: {outlier_percentage:.2f}%")


Percentage of outliers identified: 17.44%


In [ ]:
data_select = data_raw.select_dtypes(include='float')

# Initialize the One-Class SVM model
one_class_svm = OneClassSVM(kernel='rbf', gamma=0.1, nu=0.1)  # You may adjust `gamma` and `nu` as needed

# Fit the model and predict outliers
outliers = one_class_svm.fit_predict(data_select)

# Add the outlier information to the DataFrame
# In One-Class SVM, label `-1` indicates outliers
data_select['Outlier'] = np.where(outliers == -1, 'Yes', 'No')

# Calculate the overall percentage of outliers
outlier_percentage = (data_select['Outlier'] == 'Yes').mean() * 100
print(f"\nPercentage of outliers identified: {outlier_percentage:.2f}%")